In [14]:
#import relevant libraries
import numpy as np
import pandas as pd
import random as random
import matplotlib.pyplot as plt
import os
from PIL import Image
import keras as K
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation
from keras.layers import Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

print('Libraries imported succesfully')

Libraries imported succesfully


In [15]:
DIRNAME_train = './datasets/training_set'
DIRNAME_test = './datasets/test_set'

In [16]:
IMG_HEIGHT = 64
IMG_WIDTH = 64

In [17]:
train_images = []
for dirname, _, filenames in os.walk(DIRNAME_train):
    for filename in filenames:   
        train_images.append(os.path.join(dirname, filename))
random.shuffle(train_images) 



In [18]:
train_datasets=[]       
for images in train_images:
    if 'DS_Store' in images:
        break
    elif 'dog' in images:
        label=0
    else:
        label=1    
    img = Image.open(images)
    img = img.resize((IMG_HEIGHT, IMG_WIDTH))
    img_array= np.asarray(img)
    train_datasets.append([label,img_array])
            
random.shuffle(train_datasets)
m=len(train_datasets)
print(m)

3721


In [19]:

df=pd.DataFrame(train_datasets)
df.columns = ["label", "ImageArray"]
df.head(-10)


,label,ImageArray
0,0,"[[[248, 251, 246], [245, 248, 239], [237, 240,..."
1,0,"[[[105, 99, 77], [96, 89, 70], [90, 79, 64], [..."
2,0,"[[[82, 97, 134], [31, 43, 69], [37, 47, 56], [..."
3,1,"[[[41, 46, 50], [44, 49, 53], [48, 52, 57], [5..."
4,1,"[[[58, 50, 38], [60, 55, 46], [62, 62, 63], [5..."
...,...,...
3706,0,"[[[14, 16, 15], [17, 19, 18], [16, 18, 17], [1..."
3707,1,"[[[217, 226, 228], [193, 198, 196], [239, 242,..."
3708,0,"[[[82, 83, 83], [90, 81, 85], [96, 80, 87], [9..."
3709,0,"[[[154, 106, 118], [146, 100, 106], [147, 98, ..."


In [20]:
print(df.shape)

(3721, 2)


In [21]:
for i in range(10):
    print("Input Shape : ")
    print(df.ImageArray[i].shape)
    print(" Output data")
    print(df.label[i])

Input Shape : 
(64, 64, 3)
 Output data
0
Input Shape : 
(64, 64, 3)
 Output data
0
Input Shape : 
(64, 64, 3)
 Output data
0
Input Shape : 
(64, 64, 3)
 Output data
1
Input Shape : 
(64, 64, 3)
 Output data
1
Input Shape : 
(64, 64, 3)
 Output data
1
Input Shape : 
(64, 64, 3)
 Output data
0
Input Shape : 
(64, 64, 3)
 Output data
1
Input Shape : 
(64, 64, 3)
 Output data
1
Input Shape : 
(64, 64, 3)
 Output data
0


In [22]:
input = []
output = []

for data in train_datasets:
    input.append(data[1])
    output.append([data[0]])  
    
input = np.array(input)
output = np.array(output)

print("Input Shape : {}".format(input.shape))
print("Output Shape : {}".format(output.shape))
 

Input Shape : (3721, 64, 64, 3)
Output Shape : (3721, 1)


In [23]:
input_shape=(64,64,3)
def model():
    model = K.Sequential()
    
    model.add(Conv2D(32 , kernel_size=(3, 3),  input_shape = input_shape, padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu')) 
    model.add(MaxPooling2D(pool_size  = (2,2)))
    
    model.add(Conv2D(64 , kernel_size=(3, 3),  input_shape = input_shape, padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size  = (2,2)))
    
    model.add(Conv2D(128 , kernel_size=(3, 3),  input_shape = input_shape, padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size  = (2,2)))
    
    
    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
  
    return model
    
    

In [24]:
model = model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
batch_normalization_3 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
activation_3 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_4 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
activation_4 (Activation)    (None, 32, 32, 64)       

In [25]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(input, output, batch_size=32, epochs=12, validation_split=0.1)

Epoch 1/12
105/105 [==============================] - 14s 126ms/step - loss: 1.7675 - accuracy: 0.5592 - val_loss: 0.7354 - val_accuracy: 0.5550
Epoch 2/12
105/105 [==============================] - 13s 126ms/step - loss: 0.5640 - accuracy: 0.7226 - val_loss: 0.6915 - val_accuracy: 0.5818
Epoch 3/12
105/105 [==============================] - 14s 129ms/step - loss: 0.5375 - accuracy: 0.7256 - val_loss: 0.5645 - val_accuracy: 0.7105
Epoch 4/12
105/105 [==============================] - 13s 128ms/step - loss: 0.4589 - accuracy: 0.7913 - val_loss: 1.2299 - val_accuracy: 0.5576
Epoch 5/12
105/105 [==============================] - 13s 126ms/step - loss: 0.4160 - accuracy: 0.8093 - val_loss: 0.5494 - val_accuracy: 0.7292
Epoch 6/12
105/105 [==============================] - 13s 126ms/step - loss: 0.3778 - accuracy: 0.8353 - val_loss: 0.8541 - val_accuracy: 0.6193
Epoch 7/12
105/105 [==============================] - 13s 127ms/step - loss: 0.3177 - accuracy: 0.8569 - val_loss: 0.5520 - val_ac